Load articles and prune ones without abstracts, since we're using the abstracts for generating the embeddings.

In [ ]:
from tinydb import TinyDB, Query

db = TinyDB("db.json")
table = db.table("articles")

articles = table.all()
print(f"loaded {len(articles)} articles")

articles = [x for x in articles if x["abstract"] != "No abstract available."]
print(f"retaining {len(articles)} articles")

Stage the articles so that they can easily be loaded into the vector database.

In [ ]:
documents = []
ids = []

for article in articles:
    doc_id = article["link"]
    if doc_id not in ids:
        documents.append(article["abstract"])
        ids.append(doc_id)

print(f"loaded {len(ids)} articles")

For finding semantically related documents, we'll use Chroma (https://www.trychroma.com/), which is a lightweight vector data store. Chroma supports swappable embedding models, filtering using metadata, keyword search, and multiple distance measurements. We'll use these features for evlauating approaches to organizing papers for downstream processing (search, summarization, keyword extraction, etc.).

In [ ]:
import chromadb
from chromadb.utils import embedding_functions

client = chromadb.PersistentClient(path="vectors_db")

In [ ]:
from chromadb.errors import InvalidCollectionException

collection_name = "articles-specter-embeddings"
ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="allenai-specter")

try:
    collection = client.get_collection(name=collection_name, embedding_function=ef)

    print(f"loaded collection {collection_name}")
except InvalidCollectionException:
    print(f"creating collection {collection_name}")

    collection = client.create_collection(name=collection_name, embedding_function=ef)

In [ ]:
collection.add(documents=documents, ids=ids)

In [ ]:
results = collection.query(
    query_texts=["infectious diseases transmitted by mosquitoes and that affect children"],
    n_results=10,
)

results